In [2]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import OLSInfluence, variance_inflation_factor
import scipy as sci
from scipy.stats import f
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [12]:
df = pd.read_excel('/content/Dados.xlsx')

In [13]:
df.head()

,ID,"Número médio de viagens por ano (ex: 1, 2, 3...)",Orçamento médio para uma viagem em Real (ex: 1000 reais),"Duração média de uma viagem em dias (ex: 1,2,3...)",Tipo de destino preferido,Principal motivo da viagem
0,1,3,5000,6,Cidade,Lazer
1,2,2,30000,20,Outro,Lazer
2,3,2,5000,5,Natureza,Lazer
3,4,1,4000,7,Praia,Lazer
4,5,1,5000,8,Praia,Lazer


In [42]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Remover espaços em branco dos nomes das colunas
df.columns = df.columns.str.strip()

# Verificar se as colunas categóricas estão presentes
print(df.columns)

# Transformar variáveis categóricas em variáveis dummy
df_dummies = pd.get_dummies(df, columns=['Tipo de destino preferido', 'Principal motivo da viagem'], drop_first=True)

# Converter colunas booleanas para numéricas
df_dummies = df_dummies.applymap(lambda x: 1 if x is True else (0 if x is False else x))

# Garantir que todas as colunas são do tipo numérico
df_dummies = df_dummies.apply(pd.to_numeric)

# Verificar os tipos de dados das colunas
print(df_dummies.dtypes)

# Separar as variáveis dependentes e independentes
X = df_dummies.drop(columns=['ID', 'Orçamento médio para uma viagem em Real (ex: 1000 reais)'])
y = df_dummies['Orçamento médio para uma viagem em Real (ex: 1000 reais)']


# Função para ajustar o modelo e remover variáveis não significativas
def ajustar_modelo(X, y):
    X = sm.add_constant(X)
    modelo = sm.OLS(y, X).fit()
    while True:
        pvalues = modelo.pvalues
        max_pvalue = pvalues.max()
        if max_pvalue > 0.05:
            excluded_feature = pvalues.idxmax()
            print(f"Removendo variável não significativa: {excluded_feature} com p-valor = {max_pvalue}")
            X = X.drop(columns=[excluded_feature])
            modelo = sm.OLS(y, X).fit()
        else:
            break
    return modelo

# Ajustar o modelo e remover variáveis não significativas
modelo = ajustar_modelo(X, y)

# Mostrar o sumário final do modelo
print(modelo.summary(title='Sumário do Modelo de Regressão Linear'))

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Treinar o modelo de regressão linear usando sklearn
model = LinearRegression()
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Mostrar as previsões
print("Previsões no conjunto de teste:")
print(y_pred)

Index(['ID', 'Número médio de viagens por ano (ex: 1, 2, 3...)',
       'Orçamento médio para uma viagem em Real (ex: 1000 reais)',
       'Duração média de uma viagem em dias (ex: 1,2,3...)',
       'Tipo de destino preferido', 'Principal motivo da viagem'],
      dtype='object')
ID                                                          int64
Número médio de viagens por ano (ex: 1, 2, 3...)            int64
Orçamento médio para uma viagem em Real (ex: 1000 reais)    int64
Duração média de uma viagem em dias (ex: 1,2,3...)          int64
Tipo de destino preferido_Montanha                          int64
Tipo de destino preferido_Natureza                          int64
Tipo de destino preferido_Outro                             int64
Tipo de destino preferido_Praia                             int64
Principal motivo da viagem_Cultural                         int64
Principal motivo da viagem_Lazer                            int64
Principal motivo da viagem_Negócios                       

In [43]:
# Criando uma tabela com y_test e y_pred para comparar os resultados
result_df = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred})


In [39]:
result_df.head()

,y_test,y_pred
22,5000,5319.261004
20,3000,12918.681307
25,10000,11248.751442
4,5000,3405.671141
10,5000,787.078999


In [44]:
# Avaliando o modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [45]:
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 30858713.25138626
R-squared: -2.857483812066235
